In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 160772 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.26-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.26-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.26-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Sun Jun  6 07:40:54 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!mkdir -p drive
!google-drive-ocamlfuse -o nonempty drive
import os
import sys
os.chdir('drive')

In [4]:
import nltk
from functools import lru_cache
import re
import string
import numpy as np
import pandas as pd
import torch
from collections import Counter, defaultdict
from sklearn.preprocessing import OneHotEncoder
import torch.utils.data as Data
import torch.nn.functional as F
import torch
import torch.nn as nn
from torch.autograd import Variable
import os
import glob
from sklearn.utils import shuffle
import copy

In [5]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    # print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print(torch.cuda.device_count())
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

1
Device name: Tesla P100-PCIE-16GB


In [6]:
!/opt/bin/nvidia-smi

Sun Jun  6 07:40:59 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    26W / 250W |      2MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [7]:
class Preprocessor:
    def clean_text(self, text):
        # '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
        # make text lowercase
        text1 = text.lower()
        # remove square brackets
        text1 = re.sub('\[.*?\]', '', text1)
        #remove <>
        text1 = re.sub('<.*?>+', '', text1)
        text1 = re.sub('\(.*?\)', ' ', text1)
        text1 = re.sub('\{.*?\}', ' ', text1)
        # remove links
        text1 = re.sub('https?://\S+|www\.\S+', ' ', text1)
        # remove punctuation
        text1 = re.sub('[%s]' % re.escape(string.punctuation), '', text1)
        # remove \n
        # text = re.sub('\n', '', text)
        # remove numbers
        text = re.sub('\w*\d\w*', '', text)
        return text1

    def __init__(self):
        self.stem = lru_cache(maxsize=10000)(nltk.stem.SnowballStemmer('english').stem)
        # self.stopwords = stopwords.words('english')
        self.tokenize = nltk.tokenize.TreebankWordTokenizer().tokenize

    def __call__(self, text):
        text1 = self.clean_text(text)
        tokens = self.tokenize(text1)
        # tokens = [token for token in tokens if token not in self.stopwords]
        # tokens = [self.stem(token) for token in tokens]

        return tokens

In [8]:

class DataLoader:
    def __init__(self, data, batch_size, k, context_window = 1, preprocessor=Preprocessor(), enc_tokens = OneHotEncoder(), dev = 'cpu'):
        self.df = data
        self.dev = dev
        self.batch_size = batch_size
        self.context_window = context_window
        self.padding_length = 75
        self.fold_num = 10
        # self.padding_length = 192
        # self.first_sentence = 128
        # self.second_sentence = self.padding_length - self.first_sentence

        self.apply_preprocessor(preprocessor)
        enc_tokens.fit(self.vocab)
        self.split(0.9, 0.1)

        self.k_fold_partition(fold_num = self.fold_num) # Divide ten folds
        print("K Fold Partition Completed!")
        train, validation = self.k_fold_split(k)
        trainX, trainY = self.build_training_data(enc_tokens, train)
        validationX, validationY = self.build_training_data(enc_tokens, validation)
        testX, testY = self.build_training_data(enc_tokens, self.test)
        print(f'Train Dataset Shape : {trainX.shape}')
        print(f'Validation Dataset Shape : {validationX.shape}')
        print(f'Test Dataset Shape : {testX.shape}')
        self.train_dataset = self.get_batch(trainX, trainY)
        self.validation_dataset = self.get_batch(validationX, validationY)
        self.test_dataset = self.get_batch(testX, testY)


    def get_batch(self, X, Y):
        dataset = Data.TensorDataset(X, Y)
        loader = Data.DataLoader(
            dataset=dataset,
            batch_size=self.batch_size,
            shuffle=True,
            # num_workers=2,
        )
        return loader


    def remove_low_frequency(self, list):
        new_list = []
        for x in list:
            if x in self.token_to_count.keys():
                new_list.append(x)
        return new_list


    def apply_preprocessor(self, preprocessor):
        self.df['tokens'] = [preprocessor(s) for s in self.df['sentence']]
        self.df['tokens'] = [x[:self.padding_length]  if len(x) > self.padding_length else x for x in self.df['tokens']]

        # for index, row in self.df.iterrows():
        #     if len(row['tokens']) > self.max_length:
        #         row[toke]

        self.token_to_count = Counter([x for l in self.df['tokens'] for x in l])

        # tmp_token_to_count = self.token_to_count.copy()
        # for index, value in tmp_token_to_count.items():
        #     if value <= self.vocab_frequency:
        #         self.token_to_count.pop(index)
        # self.df['tokens'] = [self.remove_low_frequency(x) for x in self.df['tokens']]
        self.max_length = self.get_max_length()
        print(f'Max Length : {self.max_length}')

        self.vocab = list([[term] for term in self.token_to_count.keys()])

        print(f'Vocab Size : {len(self.vocab)}')

    def get_max_length(self):
        max_length = 0
        for index, row in self.df.iterrows():
            token_list = [x for x in row['tokens']]
            tmp_length = len(token_list)
            if tmp_length > max_length:
                max_length = tmp_length
        return max_length
    def k_fold_partition(self, fold_num = 10):
        # index = list(range(len(self.df))) # total index
        batch_size = int(len(self.train) / fold_num) # the number of data for each fold
        remain_num = len(self.train) - batch_size * fold_num # the remain data after partition
        self.fold_data = []
        fold_batch_list = [] # Average the remaining data to the folds
        for fold in range(fold_num):
          if remain_num > 0:
            remain_num -= 1
            fold_batch_list.append(batch_size + 1)
          else:
            fold_batch_list.append(batch_size)
        fold_index = 0 # The starting position of each division data
        for fold in range(fold_num):
          fold_texts = [fold_index, fold_index + fold_batch_list[fold]]
          self.fold_data.append(fold_texts)
          fold_index = fold_index + fold_batch_list[fold]


    def k_fold_split(self, k):
        print(f'K : {k}, fold_data[k] : {self.fold_data[k]}')
        validation = self.train.iloc[self.fold_data[k][0]: self.fold_data[k][1]]
        train = self.train.iloc[0: self.fold_data[k][0]].append(self.train.iloc[self.fold_data[k][1]:])
        train_num = len(self.train) - (self.fold_data[k][1] - self.fold_data[k][0])
        if train_num % self.batch_size == 0:
            self.no_batch = train_num / self.batch_size
        else:
            self.no_batch = int(train_num / self.batch_size) + 1
        print(f' index : {self.no_batch}')
        return train, validation


    def split(self, train, test):
        index = int(train * len(self.df))
        self.train = self.df.iloc[0:index]
        self.test = self.df.iloc[index:]

    def build_training_data(self, enc, df):
        X = []
        Y = []
        for index, row in df.iterrows():
            tmp = [enc.transform([[t]]).toarray()[0] for t in row['tokens']]
            if len(tmp) < self.max_length:
                pad_length = self.max_length - len(tmp)
                for i in range(pad_length):
                    tmp.append(np.zeros(len(self.vocab)))
            X.append(tmp)
            Y.append(row['label'])
        X = np.array(X)
        Y = np.array(Y)
        X = torch.from_numpy(X).float()
        Y = torch.from_numpy(Y).long()
        print(f'Input Data Shape (sequence_num, sequence_len, vocab_size + 1) : {X.shape}')
        print(f'Input Label Shape : {Y.shape}')
        return X, Y


In [13]:
class RelexNet(nn.Module):
    #Single step RNN.
    #input_size is char_vacab_size=26,hidden_size is number of hidden neurons，output_size is number of categories
    def __init__(self, vocabulary_size, len_seq, num_classes):
        super(RelexNet, self).__init__()
        self.num_classes = num_classes
        self.L = nn.Linear(vocabulary_size, num_classes)
        self.dropout = nn.Dropout(p=0.5)
        self.O = nn.Softmax(dim=1)

    def forward(self, input, B_layer):
        # X.shape = (batch, seq_len, vocab_size)
        T = input.shape[1]
        batch = input.shape[0]
        predict_y = Variable(torch.zeros(batch, self.num_classes))

        if B_layer is None:
            B_layer = Variable(torch.zeros(batch, self.num_classes)).cuda()

        for t in range(T):
            tmp = input[:, t, :]

            L_onestep = self.L(tmp)
            L_onestep = self.dropout(L_onestep)
            L_onestep = torch.sigmoid(L_onestep)
            # L_onestep = F.relu6(L_onestep)
           
            B_layer = torch.add(B_layer, L_onestep)
            # print(B_layer)

            if self.num_classes == 1:
                O_layer[t] = F.sigmoid(B_layer)
            else:
                O_layer = self.O(B_layer)

        return O_layer, B_layer


In [14]:
def train_model(dataset, model, optimizer, scheduler, num_epochs, dev):
    losses = []
    for epoch in range(num_epochs):
        # training mode
        # dataset.set_partition(dataset.train)
        model.train()
        total_train_loss = 0
        total_train_correct = 0
        count = 0
        for x, y in dataset.train_dataset:
            # for every batch in the training dataset perform one update step of the optimizer.
            state = None
            model.zero_grad()
            y_h, state = model(x.to(dev), state)
            loss = F.cross_entropy(y_h, y.to(dev))
            optimizer.zero_grad()
            # scheduler.zero_grad()
            loss.backward()
            optimizer.step()
            scheduler.step()
            total_train_loss += loss.item()
            total_train_correct += (y_h.argmax(-1) == y.cuda()).float().mean()
            count += 1
        average_train_loss = total_train_loss / count
        average_train_accuracy = total_train_correct / count
        losses.append(average_train_loss)
        print('{} optim: {}'.format(epoch + 1, optimizer.param_groups[0]['lr']))
        # print('{} optim: {}'.format(epoch, optimizer.param_groups[0]['lr']))
        # print('{} scheduler: {}'.format(epoch, scheduler.get_lr()[0]))

        # validation mode
        model.eval()
        total_valid_loss = 0
        total_valid_correct = 0

        count = 0
        for x, y in dataset.test_dataset:
            state = None
            y_h, state = model(x.to(dev), state)
            loss = F.cross_entropy(y_h, y.to(dev))

            total_valid_loss += loss.item()
            total_valid_correct += (y_h.argmax(-1) == y.cuda()).float().mean()
            count += 1
        average_valid_loss = total_valid_loss / count
        losses.append((average_train_loss, average_valid_loss))
        average_valid_accuracy = total_valid_correct / count
        print(f'epoch {epoch + 1} accuracies: \t train: {average_train_accuracy}\t valid: {average_valid_accuracy} loss: {average_train_loss}\t')

    # test mode
    # dataset.set_partition(dataset.test)
    model.eval()
    total_test_correct = 0
    count = 0
    for x, y in dataset.test_dataset:
        state = None
        y_h, state = model(x.to(dev), state)
        total_test_correct += (y_h.argmax(-1) == y.cuda()).float().mean()
        count += 1
    average_test_accuracy = total_test_correct / count

    print(f'test accuracy {average_test_accuracy}')

    return losses, (average_train_accuracy, average_valid_accuracy, average_test_accuracy)

In [15]:
num_epochs = 20
batch_size = 32  # Number of hidden neurons in model
context_window = 1
k = 0

dev = 'cuda' if torch.cuda.is_available() else 'cpu'  # If you have a GPU installed, use that, otherwise CPU
print(dev)
print('Loading data...')
all_files = glob.glob("data/sentiment/*.csv")
data = pd.concat((pd.read_csv(f, header=None, index_col=None) for f in all_files))
data.columns = ['sentence', 'label']
data = data.sample(frac=1, random_state=1)
dataset = DataLoader(data, k=k, batch_size = batch_size, context_window=context_window, dev=dev)
print("Data Ready!")
vocabulary_size = len(dataset.vocab)
len_seq = dataset.max_length
num_classes = 2

model = RelexNet(vocabulary_size=vocabulary_size, len_seq=len_seq, num_classes=num_classes).cuda()
# optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
optimizer = torch.optim.Adam(model.parameters(),lr=0.01,betas=(0.9,0.99))
# optimizer = torch.optim.Adam(model.parameters(),lr=0.01,betas=(0.9,0.99), weight_decay=10)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer,milestones=[8 * dataset.no_batch, 14 * dataset.no_batch],gamma = 0.1, last_epoch=-1)

losses, accuracies = train_model(dataset, model, optimizer, scheduler, num_epochs, dev=dev)
# torch.save(model, os.path.join('classifier.pth'))

cuda
Loading data...
Max Length : 70
Vocab Size : 5309
K Fold Partition Completed!
K : 0, fold_data[k] : [0, 270]
 index : 76
Input Data Shape (sequence_num, sequence_len, vocab_size + 1) : torch.Size([2430, 70, 5309])
Input Label Shape : torch.Size([2430])
Input Data Shape (sequence_num, sequence_len, vocab_size + 1) : torch.Size([270, 70, 5309])
Input Label Shape : torch.Size([270])
Input Data Shape (sequence_num, sequence_len, vocab_size + 1) : torch.Size([300, 70, 5309])
Input Label Shape : torch.Size([300])
Train Dataset Shape : torch.Size([2430, 70, 5309])
Validation Dataset Shape : torch.Size([270, 70, 5309])
Test Dataset Shape : torch.Size([300, 70, 5309])
Data Ready!
1 optim: 0.01
epoch 1 accuracies: 	 train: 0.5544407963752747	 valid: 0.6104166507720947 loss: 0.6827591767436579	
2 optim: 0.01
epoch 2 accuracies: 	 train: 0.7395833134651184	 valid: 0.659375011920929 loss: 0.6386701150944358	
3 optim: 0.01
epoch 3 accuracies: 	 train: 0.7809210419654846	 valid: 0.69791668653488

The code below is used for test on stance detection dataset before, 
which is not used at all

In [ ]:
# Abortion
num_epochs = 20
batch_size = 32  # Number of hidden neurons in model
context_window = 1

dev = 'cuda' if torch.cuda.is_available() else 'cpu'  # If you have a GPU installed, use that, otherwise CPU
print(dev)
print('Loading data...')
# root_path = 'data/stance'
# files = ['abortion', 'gayRights', 'marijuana', 'obama']
# label = {'abortion' : 0, 'gayRights' : 1, 'marijuana' : 2, 'obama' : 3}
# list_sentence = []
# list_label = []
# for file in files:
#     path = os.path.join('data', 'stance', file, '*.data')
#     print(path)
#     all_files = glob.glob(path)
#     # print(all_files)
#     for label_file in all_files:
#         f = open(label_file, 'r', encoding='UTF-8')
#         tmp = f.read()
#         list_sentence.append(tmp)
#         list_label.append(label[file])

# dict = {'sentence': list_sentence, 'label': list_label}
# data = pd.DataFrame(dict)
# data = data.sample(frac=1, random_state=1)

data = pd.read_csv("abortion.csv", index_col=None, error_bad_lines=False, encoding='UTF-8')
print(len(data))
# print(data)
data = data.sample(frac=1, random_state=256)
# data = shuffle(data)
dataset = DataLoader(data, batch_size = batch_size, context_window=context_window, dev=dev)
# print(f'Data Size: {dataset.df.size()}')
print("Data Ready!")
vocabulary_size = len(dataset.vocab)
len_seq = dataset.max_length
num_classes = 2

model = RelexNet(vocabulary_size=vocabulary_size, len_seq=len_seq, num_classes=num_classes).cuda()
# optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
optimizer = torch.optim.Adam(model.parameters(),lr=0.001,betas=(0.9,0.99))
# optimizer = torch.optim.Adam(model.parameters(),lr=0.01,betas=(0.9,0.99), weight_decay=10)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer,milestones=[10 * dataset.no_batch, 15 * dataset.no_batch],gamma = 0.1, last_epoch=-1)

losses, accuracies = train_model(dataset, model, optimizer, scheduler, num_epochs, dev=dev)
torch.save(model, os.path.join('saved_models', 'classifier_abortion.pth'))

cuda
Loading data...
1915
Max Length : 192
Vocab Size : 2093
 index : 24
Input Data Shape (sequence_num, sequence_len, vocab_size + 1) : torch.Size([766, 192, 2093])
Input Label Shape : torch.Size([766])
Input Data Shape (sequence_num, sequence_len, vocab_size + 1) : torch.Size([574, 192, 2093])
Input Label Shape : torch.Size([574])
Input Data Shape (sequence_num, sequence_len, vocab_size + 1) : torch.Size([575, 192, 2093])
Input Label Shape : torch.Size([575])
Data Ready!
1 optim: 0.001
epoch 1 accuracies: 	 train: 0.5665798783302307	 valid: 0.5452961921691895	 valid loss: 0.6875425945798694	 precision: 0.3528748590755355	 recall: 0.5452961672473867	
2 optim: 0.001
epoch 2 accuracies: 	 train: 0.5754340887069702	 valid: 0.5452961921691895	 valid loss: 0.6877200154373455	 precision: 0.3528748590755355	 recall: 0.5452961672473867	
3 optim: 0.001
epoch 3 accuracies: 	 train: 0.5666667222976685	 valid: 0.5452961921691895	 valid loss: 0.6841082919246229	 precision: 0.3528748590755355	 reca

In [ ]:
# GayRights
num_epochs = 20
batch_size = 32  # Number of hidden neurons in model
context_window = 1

dev = 'cuda' if torch.cuda.is_available() else 'cpu'  # If you have a GPU installed, use that, otherwise CPU
print(dev)
print('Loading data...')

data = pd.read_csv("gayRights.csv", index_col=None, error_bad_lines=False, encoding='UTF-8')
print(len(data))
# print(data)
data = data.sample(frac=1, random_state=1)
# data = shuffle(data)
dataset = DataLoader(data, batch_size = batch_size, context_window=context_window, dev=dev)
# print(f'Data Size: {dataset.df.size()}')
print("Data Ready!")
vocabulary_size = len(dataset.vocab)
len_seq = dataset.max_length
num_classes = 2

# model = FastText(len(dataset.token_to_id)+2, num_hidden, len(dataset.class_to_id)).to(dev)
model = RelexNet(vocabulary_size=vocabulary_size,len_seq=len_seq, num_classes=num_classes).cuda()
# optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
optimizer = torch.optim.Adam(model.parameters(),lr=0.001,betas=(0.9,0.99))
# optimizer = torch.optim.Adam(model.parameters(),lr=0.01,betas=(0.9,0.99), weight_decay=10)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer,milestones=[10 * dataset.no_batch, 15 * dataset.no_batch],gamma = 0.1, last_epoch=-1)

losses, accuracies = train_model(dataset, model, optimizer, scheduler, num_epochs, dev=dev)
torch.save(model, os.path.join('saved_models', 'classifier_gayRights.pth'))

cuda
Loading data...
1376
Max Length : 188
Vocab Size : 1629
 index : 18
Input Data Shape (sequence_num, sequence_len, vocab_size + 1) : torch.Size([550, 188, 1629])
Input Label Shape : torch.Size([550])
Input Data Shape (sequence_num, sequence_len, vocab_size + 1) : torch.Size([412, 188, 1629])
Input Label Shape : torch.Size([412])
Input Data Shape (sequence_num, sequence_len, vocab_size + 1) : torch.Size([414, 188, 1629])
Input Label Shape : torch.Size([414])
Data Ready!
1 optim: 0.001
epoch 1 accuracies: 	 train: 0.4548611044883728	 valid: 0.6286407709121704	 valid loss: 0.66835132429322	 precision: 0.3859910581222057	 recall: 0.6286407766990292	
2 optim: 0.001
epoch 2 accuracies: 	 train: 0.6452546119689941	 valid: 0.6286407709121704	 valid loss: 0.6616488872367201	 precision: 0.3859910581222057	 recall: 0.6286407766990292	
3 optim: 0.001
epoch 3 accuracies: 	 train: 0.6452546119689941	 valid: 0.6286407709121704	 valid loss: 0.6611766978664305	 precision: 0.3859910581222057	 recall

In [ ]:
# Marijuana
num_epochs = 20
batch_size = 32  # Number of hidden neurons in model
context_window = 1

dev = 'cuda' if torch.cuda.is_available() else 'cpu'  # If you have a GPU installed, use that, otherwise CPU
print(dev)
print('Loading data...')

data = pd.read_csv("marijuana.csv", index_col=None, error_bad_lines=False, encoding='UTF-8')
print(len(data))
# print(data)
data = data.sample(frac=1, random_state=1)
# data = shuffle(data)
dataset = DataLoader(data, batch_size = batch_size, context_window=context_window, dev=dev)
# print(f'Data Size: {dataset.df.size()}')
print("Data Ready!")
vocabulary_size = len(dataset.vocab)
len_seq = dataset.max_length
num_classes = 2

# model = FastText(len(dataset.token_to_id)+2, num_hidden, len(dataset.class_to_id)).to(dev)
model = RelexNet(vocabulary_size=vocabulary_size,len_seq=len_seq, num_classes=num_classes).cuda()
# optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
optimizer = torch.optim.Adam(model.parameters(),lr=0.001,betas=(0.9,0.99))
# optimizer = torch.optim.Adam(model.parameters(),lr=0.01,betas=(0.9,0.99), weight_decay=10)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer,milestones=[10 * dataset.no_batch, 15 * dataset.no_batch],gamma = 0.1, last_epoch=-1)

losses, accuracies = train_model(dataset, model, optimizer, scheduler, num_epochs, dev=dev)
torch.save(model, os.path.join('saved_models', 'classifier_marijuana.pth'))

cuda
Loading data...
626
Max Length : 182
Vocab Size : 866
 index : 8
Input Data Shape (sequence_num, sequence_len, vocab_size + 1) : torch.Size([250, 182, 866])
Input Label Shape : torch.Size([250])
Input Data Shape (sequence_num, sequence_len, vocab_size + 1) : torch.Size([187, 182, 866])
Input Label Shape : torch.Size([187])
Input Data Shape (sequence_num, sequence_len, vocab_size + 1) : torch.Size([189, 182, 866])
Input Label Shape : torch.Size([189])
Data Ready!
1 optim: 0.001
epoch 1 accuracies: 	 train: 0.7454928159713745	 valid: 0.7058823704719543	 valid loss: 0.606004017718973	 precision: 0.41379310344827586	 recall: 0.7058823529411765	
2 optim: 0.001
epoch 2 accuracies: 	 train: 0.7427884340286255	 valid: 0.7058823704719543	 valid loss: 0.6059681193076353	 precision: 0.41379310344827586	 recall: 0.7058823529411765	
3 optim: 0.001
epoch 3 accuracies: 	 train: 0.7463942170143127	 valid: 0.7058823704719543	 valid loss: 0.6061013513070377	 precision: 0.41379310344827586	 recall: 

In [ ]:
# Obama
num_epochs = 20
batch_size = 32  # Number of hidden neurons in model
context_window = 1

dev = 'cuda' if torch.cuda.is_available() else 'cpu'  # If you have a GPU installed, use that, otherwise CPU
print(dev)
print('Loading data...')

data = pd.read_csv("obama.csv", index_col=None, error_bad_lines=False, encoding='UTF-8')
print(len(data))
# print(data)
data = data.sample(frac=1, random_state=1)
# data = shuffle(data)
dataset = DataLoader(data, batch_size = batch_size, context_window=context_window, dev=dev)
# print(f'Data Size: {dataset.df.size()}')
print("Data Ready!")
vocabulary_size = len(dataset.vocab)
len_seq = dataset.max_length
num_classes = 2

# model = FastText(len(dataset.token_to_id)+2, num_hidden, len(dataset.class_to_id)).to(dev)
model = RelexNet(vocabulary_size=vocabulary_size, len_seq=len_seq, num_classes=num_classes).cuda()
# optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
optimizer = torch.optim.Adam(model.parameters(),lr=0.001,betas=(0.9,0.99))
# optimizer = torch.optim.Adam(model.parameters(),lr=0.01,betas=(0.9,0.99), weight_decay=10)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer,milestones=[10 * dataset.no_batch, 15 * dataset.no_batch],gamma = 0.1, last_epoch=-1)

losses, accuracies = train_model(dataset, model, optimizer, scheduler, num_epochs, dev=dev)
torch.save(model, os.path.join('saved_models', 'classifier_obama.pth'))

cuda
Loading data...
985
Max Length : 180
Vocab Size : 1315
 index : 13
Input Data Shape (sequence_num, sequence_len, vocab_size + 1) : torch.Size([394, 180, 1315])
Input Label Shape : torch.Size([394])
Input Data Shape (sequence_num, sequence_len, vocab_size + 1) : torch.Size([295, 180, 1315])
Input Label Shape : torch.Size([295])
Input Data Shape (sequence_num, sequence_len, vocab_size + 1) : torch.Size([296, 180, 1315])
Input Label Shape : torch.Size([296])
Data Ready!
1 optim: 0.001
epoch 1 accuracies: 	 train: 0.48125001788139343	 valid: 0.4644067883491516	 valid loss: 0.7405359450033155	 precision: 1.0	 recall: 0.46440677966101696	
2 optim: 0.001
epoch 2 accuracies: 	 train: 0.48894232511520386	 valid: 0.47457626461982727	 valid loss: 0.695393718905368	 precision: 0.5857740585774058	 recall: 0.4745762711864407	
3 optim: 0.001
epoch 3 accuracies: 	 train: 0.5177884697914124	 valid: 0.5355932116508484	 valid loss: 0.691359292854697	 precision: 0.3487858719646799	 recall: 0.53559322

In [ ]:
num_epochs = 20
batch_size = 32  # Number of hidden neurons in model
context_window = 1

dev = 'cuda' if torch.cuda.is_available() else 'cpu'  # If you have a GPU installed, use that, otherwise CPU
print(dev)
print('Loading data...')
# root_path = 'data/stance'
# files = ['abortion', 'gayRights', 'marijuana', 'obama']
# label = {'abortion' : 0, 'gayRights' : 1, 'marijuana' : 2, 'obama' : 3}
# list_sentence = []
# list_label = []
# for file in files:
#     path = os.path.join('data', 'stance', file, '*.data')
#     print(path)
#     all_files = glob.glob(path)
#     # print(all_files)
#     for label_file in all_files:
#         f = open(label_file, 'r', encoding='UTF-8')
#         tmp = f.read()
#         list_sentence.append(tmp)
#         list_label.append(label[file])

# dict = {'sentence': list_sentence, 'label': list_label}
# data = pd.DataFrame(dict)
# data = data.sample(frac=1, random_state=1)

data = pd.read_csv("stance_dataset.csv", index_col=None, error_bad_lines=False, encoding='UTF-8')
print(len(data))
# print(data)
data = data.sample(frac=1, random_state=1)
# data = shuffle(data)
dataset = DataLoader(data, batch_size = batch_size, context_window=context_window, dev=dev)
# print(f'Data Size: {dataset.df.size()}')
print("Data Ready!")
vocabulary_size = len(dataset.vocab)
len_seq = dataset.max_length
num_classes = 4

# model = FastText(len(dataset.token_to_id)+2, num_hidden, len(dataset.class_to_id)).to(dev)
model = RelexNet(vocabulary_size=vocabulary_size, enc=dataset.enc_modifiers, modifier_size=len(dataset.modifier_vocab),len_seq=len_seq, num_classes=num_classes).cuda()
# optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
optimizer = torch.optim.Adam(model.parameters(),lr=0.005,betas=(0.9,0.99))
# optimizer = torch.optim.Adam(model.parameters(),lr=0.01,betas=(0.9,0.99), weight_decay=10)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer,milestones=[3 * dataset.no_batch, 15 * dataset.no_batch],gamma = 0.9, last_epoch=-1)

losses, accuracies = train_model(dataset, model, optimizer, scheduler, num_epochs, dev=dev)
torch.save(model, os.path.join('saved_models', 'classifier2.pth'))

cuda
Loading data...
4902
Max Length : 189
Vocab Size : 3949
Modifier Size : 43
Negative Modifier Size : 29
 index : 62


KeyboardInterrupt: ignored

In [ ]:
torch.save(model, os.path.join('classifier.pth'))

In [ ]:
!pip install kora
from kora import console
console.start()  # and click link